In [1]:
import os
import gc
import re
import json
import pickle
import datetime
from tqdm import tqdm

import numpy as np
import pandas as pd
pd.options.display.max_columns = None

from typing import Union

import seaborn
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
%config InlineBackend.figure_formats = {'png', 'retina'}

from scipy.stats import linregress

from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

import lightgbm as lgb

from v01005_Baseline import *

In [12]:
VERSION = 'v01005'
SEED = 42

In [3]:
train = pd.read_pickle('/Users/rui/Documents/repositories/m5_forecasting_accuracy/v01000/features/drop_null_rows.pkl')

In [5]:
def split_train_eval_submit(df, pred_interval=28):
    latest_date = df['date'].max()
    submit_date = latest_date - datetime.timedelta(days=pred_interval)
    submit_mask = (df["date"] > submit_date)

    eval_date = latest_date - datetime.timedelta(days=pred_interval * 2)
    eval_mask = ((df["date"] > eval_date) & (df["date"] <= submit_date))

    train_mask = ((~eval_mask) & (~submit_mask))
    return df[train_mask], df[eval_mask], df[submit_mask]

In [8]:
print('\n--- Train Model ---\n')
target = 'sales'
cols_to_drop = ['id', 'wm_yr_wk', 'd', 'date'] + [target]
features = train.columns.tolist()
features = [f for f in features if f not in cols_to_drop]

train_data, eval_data, submit_data = split_train_eval_submit(train)


--- Train Model ---



In [42]:
def create_feature(df):
    print('Drop Null Rows.')
    check_cols = ['lag', 'rolling']
    cheked_regex = '|'.join(check_cols)
    target_cols = df.columns[df.columns.str.contains(cheked_regex)]
    is_contain_null_rows = df[target_cols].isnull().any(axis=1)
    return df.loc[~(is_contain_null_rows), :]